In [254]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Testing functions

In [255]:
def compare_2images(original , processed):
    plt.figure(figsize=(10,20))

    plt.subplot(121)
    plt.imshow(original, cmap='gray')
    plt.axis('off')
    plt.title("orignal")
    
    plt.subplot(122)
    plt.imshow(processed, cmap='gray')
    plt.axis('off')
    plt.title("after processing")
    plt.show()    


def try_all_images(func , process):
    images = [
        "frame1.png",
        "frame2.png",
        "frame3.png",
        "original.png",
        "real.jpg",
        "real2.jpg",
        "real3.jpg",
        "real4.png",
    ]

    for image in images:
        img = cv.imread("./shapes/"+image)
        # # for fast testing
        # cv.imwrite(f"./shapes/{process}/"+image, func(img))
        compare_2images(img , func(img))

# Trying different gaussian Blurring 

In [256]:
def gaussian(img):
    gaussian = cv.GaussianBlur( img, (3, 3), 0)
    return gaussian

In [ ]:
try_all_images(gaussian , "gaussian")

# Trying threshold to detect black shapes ❌

In [258]:
def threshold(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    filtered = gaussian(gray)
    _, thresh = cv.threshold(filtered, 110, 255, cv.THRESH_BINARY_INV)
    return thresh

In [ ]:
try_all_images(threshold , "threshold")